In [68]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

In [69]:
names = ["Naive Bayes", "QDA","Decision Tree", "Random Forest", "Neural Net", "AdaBoost","Nearest Neighbors", "Linear SVM", "RBF SVM"]
classifiers =[GaussianNB(),QuadraticDiscriminantAnalysis(),DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=5),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1)] 

In [70]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]

In [71]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]
#emotions = ["neutral",'happy','sad','fearful','angry','surprised','disgusted']
emotions = [0,1,2,3,4,5,6]
dbs = []
for i,f in enumerate(files):
    db = pd.read_csv(f)
    db['file'] = db['file'].map(lambda x: str(x)[15:])
    db['file'] = db['file'].map(lambda x: str(x)[:-4]).astype(int)
    db['Subjet'] = db['file']
    db['Emotions'] = emotions[i]
    dbs.append(db)

db = (pd.concat(dbs[i][:] for i in range(len(files))))
db = db.sort_values('file')
#df['Emotion']=df['file']
db.drop(['file'], axis=1, inplace=True)

In [72]:
#db.head()

In [73]:
features = list(db.columns[:13])
Etype={'neutral': 0, 'happy': 1,
        'sad': 2,'fearful': 3,'angry': 4,
        'surprised': 5,'disgusted': 6}

In [74]:
db_data = db[features]

In [75]:
db_target = db['Emotions']

In [76]:
X_train, X_test, y_train, y_test = train_test_split(db_data, db_target, test_size=0.2, random_state=0)

In [77]:
X_train.shape, y_train.shape

((1288, 13), (1288,))

In [78]:
X_test.shape, y_test.shape

((322, 13), (322,))

In [14]:
for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(X_train, y_train)
        scores = cross_val_score(clf, db_data, db_target, cv=10)
        print(scores)
        print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Naive Bayes
[ 0.68944099  0.62111801  0.61490683  0.65217391  0.62732919  0.63354037
  0.70186335  0.50310559  0.52173913  0.53416149]
Accuracy: 0.61 (+/- 0.13)
QDA
[ 0.72670807  0.68944099  0.68322981  0.70807453  0.66459627  0.63354037
  0.72670807  0.59006211  0.63975155  0.61490683]
Accuracy: 0.67 (+/- 0.09)
Decision Tree
[ 0.55279503  0.55900621  0.55279503  0.56521739  0.53416149  0.57142857
  0.57142857  0.44099379  0.51552795  0.39751553]
Accuracy: 0.53 (+/- 0.11)
Random Forest
[ 0.62111801  0.58385093  0.65838509  0.67080745  0.62111801  0.63354037
  0.68322981  0.49689441  0.52795031  0.50310559]
Accuracy: 0.60 (+/- 0.13)
Neural Net
[ 0.67080745  0.52173913  0.59006211  0.64596273  0.62111801  0.58385093
  0.65217391  0.49068323  0.55900621  0.53416149]
Accuracy: 0.59 (+/- 0.11)
AdaBoost
[ 0.54658385  0.49689441  0.55900621  0.58385093  0.52173913  0.54037267
  0.59627329  0.49068323  0.50931677  0.39130435]
Accuracy: 0.52 (+/- 0.11)
Nearest Neighbors
[ 0.58385093  0.57142857

In [15]:
clf = SVC(kernel='linear', C=1).fit(X_train, y_train)

In [16]:
scores = cross_val_score(clf, db_data, db_target, cv=10)

In [17]:
scores

array([ 0.72670807,  0.68944099,  0.64596273,  0.70186335,  0.62111801,
        0.62111801,  0.73913043,  0.51552795,  0.57763975,  0.54037267])

In [18]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.64 (+/- 0.15)


In [19]:
from sklearn.model_selection import ShuffleSplit
n_samples = db_data.shape[0]
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
cross_val_score(clf, db_data, db_target, cv=cv)

array([ 0.64596273,  0.65010352,  0.63561077])

In [20]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(
    db_data, db_target, test_size=0.4, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)  

0.64906832298136641

In [21]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(preprocessing.StandardScaler(), SVC(C=1))
cross_val_score(clf, db_data, db_target, cv=cv)  

array([ 0.6252588 ,  0.65010352,  0.6563147 ])

In [100]:
from keras.models import Sequential
from keras.layers import Dense

In [101]:
df_data = db_data.values

In [102]:
df_target = db_target.values

In [103]:
X_train, X_test, y_train, y_test = train_test_split(df_data, df_target, test_size=0.2, random_state=0)

In [104]:
nFeatures = X_train.shape[1]

In [105]:
X_train.shape, y_train.shape

((1288, 13), (1288,))

In [114]:
model = Sequential()
model.add(Dense(1, input_shape=(nFeatures,), activation='linear'))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mse', 'mae'])
model.fit(X_train, y_train, batch_size=4, epochs=10)

Epoch 1/10
1288/1288 [==============================] - 0s 300us/step - loss: 81.0106 - mean_squared_error: 81.0106 - mean_absolute_error: 7.0310
Epoch 2/10
1288/1288 [==============================] - 0s 270us/step - loss: 47.8545 - mean_squared_error: 47.8545 - mean_absolute_error: 5.1812
Epoch 3/10
1288/1288 [==============================] - 0s 288us/step - loss: 31.4269 - mean_squared_error: 31.4269 - mean_absolute_error: 4.3035
Epoch 4/10
1288/1288 [==============================] - 0s 291us/step - loss: 19.1027 - mean_squared_error: 19.1027 - mean_absolute_error: 3.4324
Epoch 5/10
1288/1288 [==============================] - 0s 308us/step - loss: 11.2079 - mean_squared_error: 11.2079 - mean_absolute_error: 2.7358
Epoch 6/10
1288/1288 [==============================] - 0s 309us/step - loss: 6.8415 - mean_squared_error: 6.8415 - mean_absolute_error: 2.2251
Epoch 7/10
1288/1288 [==============================] - 0s 304us/step - loss: 5.0330 - mean_squared_error: 5.0330 - mean_absol

In [115]:
model.evaluate(X_test, y_test, verbose=True)
 
Y_pred = model.predict(X_test)
 
print( y_test[:105] )
print( Y_pred[:105,0] )

322/322 [==============================] - 0s 106us/step
[0 2 5 5 4 4 3 3 0 0 2 5 2 2 3 6 0 2 2 2 4 1 2 3 6 4 6 1 5 1 6 6 1 0 5 2 1
 6 6 0 0 6 5 1 6 5 2 5 0 0 3 5 0 2 0 6 6 3 5 6 4 6 3 5 3 0 5 2 3 2 1 1 3 3
 1 6 0 6 2 3 1 4 6 3 6 2 6 2 5 0 3 4 2 3 4 1 2 2 1 2 4 0 0 0 6]
[ 3.04274964  2.30423665  2.45283818  3.35602188  2.01643443  2.4916954
  3.80829525  3.42440128  2.39008474  1.78134203  2.37755656  2.81837034
  3.26857042  2.54032063  2.60203481  2.2454617   2.98602223  2.46677923
  3.76599479  2.45642447  1.81260169  2.03013349  2.89533758  2.96203685
  2.7417357   2.96631026  2.49348044  3.51639557  3.60466743  2.82926083
  3.13466811  2.93079257  3.15185404  1.619946    4.89413023  2.40237784
  2.53554845  2.89060855  2.24780416  2.95551038  1.19763958  2.23965454
  4.73595476  2.59525752  2.46345162  3.54252362  3.55461907  3.64510846
  1.98691618  2.87121606  2.88041186  3.96967196  1.57686269  4.2185874
  2.2618475   2.79303575  2.14500117  3.52320027  3.56730962  2.23066831
 

In [125]:
error = Y_pred-y_test

In [126]:
print(error)

[[ 3.04274964  1.04274964 -1.95725036 ...,  3.04274964 -0.95725036
   1.04274964]
 [ 2.30423665  0.30423665 -2.69576335 ...,  2.30423665 -1.69576335
   0.30423665]
 [ 2.45283818  0.45283818 -2.54716182 ...,  2.45283818 -1.54716182
   0.45283818]
 ..., 
 [ 2.79338765  0.79338765 -2.20661235 ...,  2.79338765 -1.20661235
   0.79338765]
 [ 2.32718182  0.32718182 -2.67281818 ...,  2.32718182 -1.67281818
   0.32718182]
 [ 3.15378833  1.15378833 -1.84621167 ...,  3.15378833 -0.84621167
   1.15378833]]
